In [1]:
# spark 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# pipeline 
from pyspark.ml import Pipeline

# Indexer, vector, onhotencoder...
from pyspark.ml.feature import (VectorAssembler, VectorIndexer, 
                                OneHotEncoder, StringIndexer)

In [2]:

conf = SparkConf().setAppName("building a PIPELINE MODEL")
sc = SparkContext(conf=conf)
sqlCtx = SQLContext(sc)
spark = SparkSession.builder.enableHiveSupport().getOrCreate()


In [3]:
#xx = spark.read.csv("titanic_train.csv", header = True)

In [4]:
# data = sc.textFile('titanic_train.csv').map(lambda line: line.split(","))
# headers = data.first()
# traindata = data.filter(lambda row: row != headers)
# sqlContext = SQLContext(sc)
# df = sqlContext.createDataFrame(traindata, [ 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
#        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])

df = spark.read.csv("titanic_train.csv", header = True)
df = df.withColumn("Pclass", df["Pclass"].cast("float"))
df = df.withColumn("Age", df["Age"].cast("float"))
df = df.withColumn("Parch", df["Parch"].cast("float"))
df = df.withColumn("Survived", df["Survived"].cast("integer"))


In [5]:
df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: float (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: float (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|   3.0|Braund, Mr. Owen ...|  male|22.0|    1|  0.0|       A/5 21171|   7.25| null|       S|
|          2|       1|   1.0|Cumings, Mrs. Joh...|female|38.0|    1|  0.0|        PC 17599|71.2833|  C85|       C|
|          3|       1|   3.0|Heikkinen, Miss. ...|female|26.0|    0|  0.0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|   1.0|Futrelle, Mrs. Ja...|female|35.0|    1|  0.0|          113803|   53.1| C123|       S|
|          5|       0|   3.0|Allen, Mr. Willia...|  male|35.0|    0|  0.0|          373450|   8.05| null|       S|
|          6|       0|   3.0|    Moran, Mr. James|  male|null|    0|  0.0|      

In [7]:
# select working columns 
my_cols = df.select(['Survived', 'Pclass', 'Sex', 'Age','Parch'])

In [8]:
my_cols.show()

+--------+------+------+----+-----+
|Survived|Pclass|   Sex| Age|Parch|
+--------+------+------+----+-----+
|       0|   3.0|  male|22.0|  0.0|
|       1|   1.0|female|38.0|  0.0|
|       1|   3.0|female|26.0|  0.0|
|       1|   1.0|female|35.0|  0.0|
|       0|   3.0|  male|35.0|  0.0|
|       0|   3.0|  male|null|  0.0|
|       0|   1.0|  male|54.0|  0.0|
|       0|   3.0|  male| 2.0|  1.0|
|       1|   3.0|female|27.0|  2.0|
|       1|   2.0|female|14.0|  0.0|
|       1|   3.0|female| 4.0|  1.0|
|       1|   1.0|female|58.0|  0.0|
|       0|   3.0|  male|20.0|  0.0|
|       0|   3.0|  male|39.0|  5.0|
|       0|   3.0|female|14.0|  0.0|
|       1|   2.0|female|55.0|  0.0|
|       0|   3.0|  male| 2.0|  1.0|
|       1|   2.0|  male|null|  0.0|
|       0|   3.0|female|31.0|  0.0|
|       1|   3.0|female|null|  0.0|
+--------+------+------+----+-----+
only showing top 20 rows



In [9]:
# drop null data
my_final_data = my_cols.na.drop()

In [10]:
my_final_data.show()

+--------+------+------+----+-----+
|Survived|Pclass|   Sex| Age|Parch|
+--------+------+------+----+-----+
|       0|   3.0|  male|22.0|  0.0|
|       1|   1.0|female|38.0|  0.0|
|       1|   3.0|female|26.0|  0.0|
|       1|   1.0|female|35.0|  0.0|
|       0|   3.0|  male|35.0|  0.0|
|       0|   1.0|  male|54.0|  0.0|
|       0|   3.0|  male| 2.0|  1.0|
|       1|   3.0|female|27.0|  2.0|
|       1|   2.0|female|14.0|  0.0|
|       1|   3.0|female| 4.0|  1.0|
|       1|   1.0|female|58.0|  0.0|
|       0|   3.0|  male|20.0|  0.0|
|       0|   3.0|  male|39.0|  5.0|
|       0|   3.0|female|14.0|  0.0|
|       1|   2.0|female|55.0|  0.0|
|       0|   3.0|  male| 2.0|  1.0|
|       0|   3.0|female|31.0|  0.0|
|       0|   2.0|  male|35.0|  0.0|
|       1|   2.0|  male|34.0|  0.0|
|       1|   3.0|female|15.0|  0.0|
+--------+------+------+----+-----+
only showing top 20 rows



In [11]:
#  index columns 

gender_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

# one hot encoding 

# e.g.
# A B C 
# index 
# 0 1 2 
# ONE HOT ENCODE 
# [1,0,0] [0,1,0] [0,0,1]

gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')

In [12]:
# do the same op (above) on embarked column

embark_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [13]:
# transform needed columns to "feature vector" ( as spark milb model loading data style)

assembler = VectorAssembler(inputCols= ['Pclass', 'SexVec', 'Age','Parch'],
                            outputCol= 'features')


# ['Survived', 'Pclass', 'Sex', 'Age','Parch', 'Embarked']


In [14]:
assembler

VectorAssembler_47c4a18d6b6f189cc1b5

In [15]:
# ml model 

log_reg_titanic = LogisticRegression(featuresCol='features', labelCol='Survived')

In [16]:
# pipeline : stage for different steps 
# step : 
# indexing --> encode -> train 

pipeline = Pipeline(stages=[gender_indexer, #embark_indexer,
                            gender_encoder, #embark_encoder,
                            assembler, log_reg_titanic])

In [17]:
# split train, test data
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [18]:
# train the model
fit_model = pipeline.fit(train_data)

In [19]:
train_data.show()

+--------+------+------+----+-----+
|Survived|Pclass|   Sex| Age|Parch|
+--------+------+------+----+-----+
|       0|   1.0|female|25.0|  2.0|
|       0|   1.0|female|50.0|  0.0|
|       0|   1.0|  male|18.0|  0.0|
|       0|   1.0|  male|19.0|  0.0|
|       0|   1.0|  male|24.0|  0.0|
|       0|   1.0|  male|27.0|  2.0|
|       0|   1.0|  male|28.0|  0.0|
|       0|   1.0|  male|28.0|  0.0|
|       0|   1.0|  male|29.0|  0.0|
|       0|   1.0|  male|29.0|  0.0|
|       0|   1.0|  male|30.0|  0.0|
|       0|   1.0|  male|31.0|  0.0|
|       0|   1.0|  male|33.0|  0.0|
|       0|   1.0|  male|36.0|  0.0|
|       0|   1.0|  male|37.0|  0.0|
|       0|   1.0|  male|39.0|  0.0|
|       0|   1.0|  male|40.0|  0.0|
|       0|   1.0|  male|40.0|  0.0|
|       0|   1.0|  male|45.0|  0.0|
|       0|   1.0|  male|45.0|  0.0|
+--------+------+------+----+-----+
only showing top 20 rows



In [20]:
# do the prediction 